In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## 1.读取数据 

In [55]:
data_path = './data/gouxionghui_data/tea_35/data.xlsx'
data = pd.read_excel(data_path, na_values = np.nan)
target = "评论数"
predictors = [i for i in data.columns if i not in [target]]

## 2.了解数据、数据质量检查

#### 数据基本信息

In [56]:
data.head()

,标题,价格,评论数,好评率,促销,退货,品牌,产品毛重,商品产地
0,2017新茶云雾绿茶4盒共500克一杯香茶叶明前春茶毛尖散装礼盒茗茶,86.0,96000,1.00,限制,支持7天无理由退货,一杯香,125,中国大陆
1,乐品乐茶 2017新茶 茶叶 绿茶 雀舌茶 明前春茶嫩芽茶100g*1罐,59.0,32000,0.97,赠品,支持7天无理由退货,乐品乐茶,210,安徽省芜湖市
2,买2件送精美西施砂壶 印象堂茶叶 安溪正宗铁观音 经典山茶花陶瓷罐礼盒装,39.8,40000,0.97,赠品,支持7天无理由退货,印象堂,100,中国大陆
3,立顿（Lipton） 茶叶 红茶 黄牌精选红茶100包 200g （新老包装随机发货）,39.9,82000,0.97,满赠,不支持7天无理由退货,立顿（Lipton）,280,安徽省合肥市
4,乐品乐茶 2017新茶 茶叶 绿茶 信阳原产毛尖 雨前春茶125g*2罐,99.0,26000,0.97,赠品,支持7天无理由退货,乐品乐茶,450,安徽省芜湖市


In [74]:
print(data.columns)
print('*'  * 20)
print(data.shape)
print('*'  * 20)
print(data.dtypes)

Index(['标题', '价格', '评论数', '好评率', '促销', '退货', '品牌', '产品毛重', '商品产地'], dtype='object')
********************
(1403, 9)
********************
标题       object
价格      float64
评论数       int64
好评率     float64
促销       object
退货       object
品牌       object
产品毛重      int64
商品产地     object
dtype: object


#### 缺失值情况

In [60]:
data.isna().sum()

标题        0
价格        0
评论数       0
好评率       0
促销        0
退货        0
品牌        6
产品毛重      0
商品产地    516
dtype: int64

In [61]:
data[data['品牌'].isna()]

,标题,价格,评论数,好评率,促销,退货,品牌,产品毛重,商品产地
124,华源新茶叶安溪铁观音清香型500克正宗乌龙茶礼盒装高山手工茶,86.0,2300,0.98,赠品,支持7天无理由退货,NaN,500,中国大陆
165,2017年新茶叶海岸青绿茶日照绿茶春茶5盒共500g（100g/盒*5盒）高山绿茶茗茶,88.0,500,0.99,无,支持7天无理由退货,NaN,100,中国大陆
356,朴存 正山小种 红茶 茶叶武夷山桐木关散装礼盒装送礼12罐600g装,109.0,4100,0.96,限制,支持7天无理由退货,NaN,125,中国大陆
1151,杭颂 茶叶2017明前新茶龙井茶叶礼盒陶瓷罐装特级绿茶250g/2罐,439.0,200,0.99,跨店铺满减,支持7天无理由退货,NaN,2500,NaN
1310,杭颂 绿茶 龙井茶叶礼盒装 2017新茶明前特级 春茶200g/2罐,298.0,60,0.99,无,支持7天无理由退货,NaN,3000,中国大陆
1357,杭颂 茶叶绿茶 明前特级龙井茶2017新茶 茶叶礼盒 陶瓷罐装180g,268.0,40,1.00,无,支持7天无理由退货,NaN,2000,NaN


In [151]:
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 10000)

# data.loc[data['商品产地'].isna(), ['标题', '商品产地']]
data['商品产地国家'] = data[data['商品产地'] == '斯里兰卡']

,标题,价格,评论数,好评率,促销,退货,品牌,产品毛重,商品产地
481,TENTARX汀泰斯英式原装进口茶叶斯里兰卡 锡兰红茶粉 红茶包 罐装茶100g 原味锡兰红...,128.0,10,1.0,无,支持7天无理由退货,TENTARX,100,斯里兰卡
669,TENTARX汀泰斯英式原装进口茶叶斯里兰卡 锡兰红茶粉 红茶包 袋泡茶37.5g 香草味锡兰红茶,62.0,10,1.0,无,支持7天无理由退货,TENTARX,375,斯里兰卡
692,TENTARX汀泰斯英式原装进口茶叶斯里兰卡 锡兰红茶粉 红茶包 袋泡茶37.5g 原味锡兰红茶,62.0,10,1.0,无,支持7天无理由退货,TENTARX,375,斯里兰卡
940,TENTARX汀泰斯 进口红茶 锡兰红茶 斯里兰卡散茶 罐装茶叶50g,28.8,10,1.0,无,不支持7天无理由退货,TENTARX,50,斯里兰卡
1207,英菲尼（infini.t）锡兰高地红茶斯里兰卡锡兰明珠茶叶礼盒装,2000.0,14000,1.0,限制,支持7天无理由退货,英菲尼（infini.t）,2980,斯里兰卡
1388,斯里红 斯里兰卡红茶 锡兰精品红茶礼盒装 散装送礼茶叶四罐装240克,188.0,30,1.0,无,支持7天无理由退货,斯里红,780,斯里兰卡


### 缺失值填充

In [144]:
data.loc[(data['品牌'] == 'TENTARX') & (data['商品产地'].isna()), '商品产地'] = '斯里兰卡'

## 3.数据预处理

In [63]:
data.head()

,标题,价格,评论数,好评率,促销,退货,品牌,产品毛重,商品产地
0,2017新茶云雾绿茶4盒共500克一杯香茶叶明前春茶毛尖散装礼盒茗茶,86.0,96000,1.00,限制,支持7天无理由退货,一杯香,125,中国大陆
1,乐品乐茶 2017新茶 茶叶 绿茶 雀舌茶 明前春茶嫩芽茶100g*1罐,59.0,32000,0.97,赠品,支持7天无理由退货,乐品乐茶,210,安徽省芜湖市
2,买2件送精美西施砂壶 印象堂茶叶 安溪正宗铁观音 经典山茶花陶瓷罐礼盒装,39.8,40000,0.97,赠品,支持7天无理由退货,印象堂,100,中国大陆
3,立顿（Lipton） 茶叶 红茶 黄牌精选红茶100包 200g （新老包装随机发货）,39.9,82000,0.97,满赠,不支持7天无理由退货,立顿（Lipton）,280,安徽省合肥市
4,乐品乐茶 2017新茶 茶叶 绿茶 信阳原产毛尖 雨前春茶125g*2罐,99.0,26000,0.97,赠品,支持7天无理由退货,乐品乐茶,450,安徽省芜湖市


### 3.1数值型数据

In [77]:
data['价格'].describe()

count    1403.000000
mean      237.973521
std       397.907125
min         1.000000
25%        77.500000
50%       126.000000
75%       248.000000
max      6400.000000
Name: 价格, dtype: float64

In [76]:
data['评论数'].describe()

count     1403.000000
mean      2761.553100
std       6958.069551
min          0.000000
25%         70.000000
50%        400.000000
75%       2150.000000
max      96000.000000
Name: 评论数, dtype: float64

In [79]:
data['好评率'].describe()

count    1403.000000
mean        0.976636
std         0.022892
min         0.800000
25%         0.970000
50%         0.980000
75%         1.000000
max         1.000000
Name: 好评率, dtype: float64

In [80]:
data['产品毛重'].describe()

count     1403.000000
mean       564.833927
std        701.940555
min          8.000000
25%        200.000000
50%        357.000000
75%        650.000000
max      14900.000000
Name: 产品毛重, dtype: float64

### 3.2 文本、类别型数据

#### 标题

In [152]:
data['标题']

0                      2017新茶云雾绿茶4盒共500克一杯香茶叶明前春茶毛尖散装礼盒茗茶
1                    乐品乐茶 2017新茶 茶叶 绿茶 雀舌茶 明前春茶嫩芽茶100g*1罐
2                    买2件送精美西施砂壶 印象堂茶叶 安溪正宗铁观音 经典山茶花陶瓷罐礼盒装
3             立顿（Lipton） 茶叶 红茶 黄牌精选红茶100包 200g （新老包装随机发货）
4                    乐品乐茶 2017新茶 茶叶 绿茶 信阳原产毛尖 雨前春茶125g*2罐
5           【首件26元买2送1再送杯】安溪铁观音茶叶散装250g/包2017秋茶新茶清香型福建乌龙茶
6                 【当日达】七彩云南茶叶普洱茶熟茶饼庆沣祥雀之韵普洱熟茶非生茶精美礼盒装357g
7       立顿（Lipton） 茶叶 办公室组合装(红茶50包+绿茶50包）100包 200g （新老...
8                 2017新茶日照绿茶买1送1共500克 一杯香茶叶绿茶明前春茶散装礼盒浓香茶叶
9                        买1送1共500克 糯米香普洱茶 云南陈年普洱熟小沱茶 君享茶叶
10              彩程茶叶 2017年 高山古味 普洱 茶 生茶饼茶 357克/片 ，限购1饼10元
11                    2017新茶上市 乐品乐茶 茶叶 绿茶 碧螺春 明前一级125g*2罐
12                  正宗安吉白茶2017新茶一杯香明前春茶叶绿茶新茶散装珍稀白茶100g礼盒装
13                     买一送一 茶叶安溪铁观音新茶共500g礼盒装乌龙茶浓香型礼盒装雅鑫苑
14                             艺福堂 茶叶 特级祁门红茶类 祁门工夫红茶 200g
15                           特尊茶叶 绿茶 明前新茶 信阳原产5星毛尖125g*2袋
16            一杯香茶叶绿茶龙井茶2017新茶龙井买1送1共200克明前龙井茶春茶绿茶散装浓香礼盒装
17            

In [156]:
pattern = '(\d+克|\d+g)'
data['标题重量'] = data['标题'].str.extract(pattern)

#### 退货

In [64]:
data['退货'] = data['退货'].str.replace('', '').str.strip()
data['退货'] = data['退货'].str.replace('\ue616', '').str.strip()

In [65]:
data['退货'].unique()

array(['支持7天无理由退货', '不支持7天无理由退货'], dtype=object)

#### 商品产地

In [171]:
data['商品产地'].unique()

array(['', '安徽省芜湖市', '安徽省合肥市', '安徽', nan, '浙江省', ' 云南', '福建南平武夷山市',
       '浙江杭州', '云南凤庆', '浙江', '云南', '四川', '合肥', '山东枣庄', '云南西双版纳', '合肥市',
       '云南省普洱市', '福建', '安徽合肥市', '云南省大理市', '福建省宁德市', '杭州', '福建南平', '云南省',
       '福建省武夷山市', '安徽省黄山市', '云南省昆明市', '福建武夷山', '河南信阳', '广东', '福建省泉州市',
       '广东清远英德', '福建 漳州', '云南省临沧市', '上海市', '云南省大理', '福建泉州', '四川绵阳',
       '福建漳州', '波兰', '安徽池州', '江苏', '上海', '四川夹江', '四川宜宾', '江苏常州市', '安徽省',
       '斯里兰卡', '婺源', '福建省武夷山', '云南普洱', '安徽省霍山县', '云南省西双版纳', '云南大理',
       '安徽六安', '福建武夷山市', '安徽省六安市金寨县', '常州市', '云南昆明', '浙江省杭州市', '台湾',
       '福建省', ' 云南 大理', '贵州省', '北京', '福建省南平市', '福建省安溪县', '福建省南平武夷山市',
       '山东 枣庄', '四川省峨眉山', '福建省三明市', '浙江湖州市', '福建省霞浦县', '福建安溪', '淅江省杭州市',
       '泉州安溪', '福建南平武夷山', '山东', '江苏省溧阳市'], dtype=object)

In [170]:
data['商品产地'] = data['商品产地'].str.replace('中国', '')
data['商品产地'] = data['商品产地'].str.replace('大陆', '')

In [169]:
pd.Series.fillna?

#### 品牌

In [70]:
data['品牌'].unique()

array(['一杯香', '乐品乐茶', '印象堂', '立顿（Lipton）', '蓝翼', '七彩云南', '君享', '彩程茶叶',
       '雅鑫苑（yaxinyuan）', '艺福堂', '特尊（TEZUN）', '赛君王（saijunwang）', '水蜜潭',
       '梦龙韵', '闽景印象（minjingyinxiang）', '桃渊茗', '极白', '忆江南', '梅府茗家', '凤牌',
       '瀚臣', '郎品', '卢正浩', '茶缘宝贝', '盛茗世家', '大益', '莫等闲（MODENGXIAN）', '八马',
       '唐寨山', '凤鼎红（FENGDINGHONG）', '硒沁情（xiqinqing）', '小罐茶', '竹叶青', '茗然',
       '十斗郎', '谷徽韵', '良影茶业', '皇金龙（huangjinlong）', '西湖', '熙记',
       '天福茗茶（TenFu’s TEA）', '歸元茗', '英红九号', '养生密码（YANGSHENGMIMA）', '下关沱茶',
       '茶人岭', '上集', '润虎', '小燕子茶业', '峨眉雪芽', '清翠香（QINGCUIXIANG）',
       '惠聚春秋（HUIJUCHUNQIU）', '简木', '家乡妹（Jiaxiangmei）', '中茶', '梅龙虎', '润思',
       '元正', '谢裕大', nan, '十月茶社', '狮梅（shimei）', '百益佳', '天之红',
       '行吉茶业（Xing ji）', '六里轩', '正山堂', '佰思味（BSWBESTTEA）',
       '湛源红（ZHUAN YUAN HONG）', '文新（WenXin Tea）', '九雁（Jiuyan）', '老同志',
       '宝顶雪芽', '天城香', '康思园（Kangsiyuan）', '国艳', '积庆里', '宋品', '品竺',
       '老缪家（LAO MIAO JIA TEA）', '阅客（yueke）', '耕香园（GENGXIANGYUAN）',
       '峡谷沙龙（xiagushalong）', '吉顺号（JISHUNHAO）

#### 促销

* 有序类别型特征

In [72]:
data['促销'].unique()

array(['限制', '赠品', '满赠', '无', '跨店铺满减', '限购', '满折', '满减', '跨店铺满折', '抵用券',
       '换购'], dtype=object)

### 3.3 缺失值填充

### 3.4 异常值处理

In [157]:
data.head()

,标题,价格,评论数,好评率,促销,退货,品牌,产品毛重,商品产地,标题重量
0,2017新茶云雾绿茶4盒共500克一杯香茶叶明前春茶毛尖散装礼盒茗茶,86.0,96000,1.00,限制,支持7天无理由退货,一杯香,125,中国大陆,500克
1,乐品乐茶 2017新茶 茶叶 绿茶 雀舌茶 明前春茶嫩芽茶100g*1罐,59.0,32000,0.97,赠品,支持7天无理由退货,乐品乐茶,210,安徽省芜湖市,100g
2,买2件送精美西施砂壶 印象堂茶叶 安溪正宗铁观音 经典山茶花陶瓷罐礼盒装,39.8,40000,0.97,赠品,支持7天无理由退货,印象堂,100,中国大陆,NaN
3,立顿（Lipton） 茶叶 红茶 黄牌精选红茶100包 200g （新老包装随机发货）,39.9,82000,0.97,满赠,不支持7天无理由退货,立顿（Lipton）,280,安徽省合肥市,200g
4,乐品乐茶 2017新茶 茶叶 绿茶 信阳原产毛尖 雨前春茶125g*2罐,99.0,26000,0.97,赠品,支持7天无理由退货,乐品乐茶,450,安徽省芜湖市,125g
